In [1]:
pip install web3 requests python-decouple

     ---------------------------------------- 0.0/149.4 kB ? eta -:--:--
     ---------- -------------------------- 41.0/149.4 kB 991.0 kB/s eta 0:00:01
     ------------------------------------ - 143.4/149.4 kB 1.7 MB/s eta 0:00:01
     -------------------------------------- 149.4/149.4 kB 1.5 MB/s eta 0:00:00
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached multidict-6.1.0-cp312-cp312-win_amd64.whl.metadata (5.1 kB)
     ---------------------------------------- 0.0/66.9 kB ? eta -:--:--
     ---------------------------------------- 66.9/66.9 kB 3.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ------ --------------------------------- 0.2/1.3 MB 6.1 MB/s eta 0:00:01
   --------- ------------------------------ 0.3/1.3 MB 3.9 MB/s eta 0:00:01
   ------------ --------------------------- 0.4/1.3 MB 3.3 MB/s eta 0:00:01
   --------------- ------------------------ 0.5/1.3 MB 3.3 MB/s eta 0:00:01
   ------------------

curl -X POST \
 -H "Content-Type: application/json" \
 -d '{"query": "{ vaults(first: 5) { id authorizer protocolFeeController isPaused } pools(first: 5) { id vault { id } hook { id } hookConfig { id } } }", "operationName": "Subgraphs", "variables": {}}' \
 https://api.studio.thegraph.com/query/95121/balancer-vault-v1/version/latest


---


The bot will evaluate the impact of every transaction in the AAVE-wstETH pool against a decision tree and make trades for potential profits. The priority of this bot is to maximize profits. Minimum profit value is configurable, and the bot will only execute the transaction if criteria are met.

- **Rates API**: Integrate AAVE rate feed APIs.
- **DEX Spot Price**: The bot will compute the current DEX spot price.
- **Price Deviation Check**: The bot will check if the spot price deviates from the API price feed and in which direction. If there is a discrepancy, it will create appropriate buy or sell transactions to move the price back to the target.
- **DEX LP Fee Adjustment**: The quoted price will automatically accommodate the DEX LP fee.


---


In [ ]:
import requests

# Subgraph endpoint
SUBGRAPH_URL = (
    "https://api.studio.thegraph.com/query/95121/balancer-vault-v1/version/latest"
)

# GraphQL query
# query = """
# {
#   vaults(first: 5) {
#     id
#     authorizer
#     protocolFeeController
#     isPaused
#   }
#   pools(first: 5) {
#     id
#     vault {
#       id
#     }
#     hook {
#       id
#     }
#     hookConfig {
#       id
#     }
#   }
# }
# """

query = """
query MyQuery {
  pool(id: "0x0270daf4ee12ccb1abc8aa365054eecb1b7f4f6b") {
    totalShares
    name
    symbol
    tokens(first: 10) {
      volume
      symbol
      scalingFactor
      priceRate
      nestedPool {
        symbol
        name
        tokens {
          name
          symbol
          nestedPool {
            name
            symbol
            tokens {
              volume
              name
              symbol
            }
          }
          volume
        }
      }
    }
  }
}

"""


def fetch_pool_data(pool_address):
    # Request payload
    payload = {"query": query, "operationName": "Subgraphs", "variables": {}}

    # Headers
    headers = {"Content-Type": "application/json"}

    response = requests.post(SUBGRAPH_URL, json=payload, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Error: {response.status_code}, {response.text}")


# Example usage
pool_address = "0xE24Aba051aaeB07f981bD449f4B0b3250e08d4fB"
pool_data = fetch_pool_data(pool_address)
print(pool_data)

{'data': {'vaults': [{'id': '0x68ad967ae8393b722ec69db1018ec28af9a34493', 'authorizer': '0x0000000000000000000000000000000000000000', 'protocolFeeController': '0x8ac28abdadc7b3ffb7e7b6b9a03455b30117d345', 'isPaused': False}], 'pools': [{'id': '0x0270daf4ee12ccb1abc8aa365054eecb1b7f4f6b', 'vault': {'id': '0x68ad967ae8393b722ec69db1018ec28af9a34493'}, 'hook': {'id': '0x0000000000000000000000000000000000000000'}, 'hookConfig': {'id': '0x00000000000000000000000000000000000000000270daf4ee12ccb1abc8aa365054eecb1b7f4f6b'}}, {'id': '0x31e7f3a9b9c834a752887723b017397e928d9ede', 'vault': {'id': '0x68ad967ae8393b722ec69db1018ec28af9a34493'}, 'hook': {'id': '0x0000000000000000000000000000000000000000'}, 'hookConfig': {'id': '0x000000000000000000000000000000000000000031e7f3a9b9c834a752887723b017397e928d9ede'}}, {'id': '0x32fcb2892ca12d41d1132c59ce3bc31f98586284', 'vault': {'id': '0x68ad967ae8393b722ec69db1018ec28af9a34493'}, 'hook': {'id': '0x0000000000000000000000000000000000000000'}, 'hookConfig'

In [4]:
from web3 import Web3

w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:7545"))
balance = w3.eth.get_balance("0x2048eF8adB4174aC82a37398FE73744675281142")
print(w3.from_wei(balance, "ether"))

100


In [ ]:
import requests

# Etherscan API URL
ETHERSCAN_API_URL = "https://api.etherscan.io/api"

# Replace with your Etherscan API Key
ETHERSCAN_API_KEY = "H5FHQZJSC7CRGEK831FE5UWBB3CQH8XAET"


def fetch_contract_function(contract_address, function_selector):
    """
    Fetch data for a specific contract function using Etherscan's eth_call.
    """
    params = {
        "module": "proxy",
        "action": "eth_call",
        "to": contract_address,
        "data": function_selector,
        "tag": "latest",
        "apikey": ETHERSCAN_API_KEY,
    }

    try:
        response = requests.get(ETHERSCAN_API_URL, params=params)
        data = response.json()

        if "result" in data and data["result"] != "0x":
            return data["result"]
        else:
            print(
                f"Error fetching data for function selector {function_selector}: {data}"
            )
            return None
    except Exception as e:
        print(f"Error during API call: {e}")
        return None


def decode_string(hex_data):
    """
    Decode a hex string (UTF-8 encoded) from a contract call.
    """
    try:
        result = hex_data[2:]

        # Extract the offset and length (first 64 bytes are metadata)
        offset = int(result[:64], 16)
        length = int(result[64:128], 16)

        # Extract the actual JSON string in hex format
        json_hex = result[128 : 128 + (length * 2)]

        # Decode the hex to a UTF-8 string
        json_string = bytes.fromhex(json_hex).decode("utf-8")
        return json_string
    except Exception as e:
        print(f"Error decoding string: {e}")
        return None


def decode_integer(hex_data):
    """
    Decode an integer value from hex data.
    """
    try:
        return int(hex_data, 16)
    except Exception as e:
        print(f"Error decoding integer: {e}")
        return None


def main():
    # Contract address
    contract_address = "0x3de27efa2f1aa663ae5d458857e731c129069f29"

    # Function selectors
    selectors = {
        "name": "0x06fdde03",  # keccak256("name()")[:4]
        "getActualSupply": "0x18160ddd",  # Replace if actual selector differs
        "totalSupply": "0x18160ddd",  # keccak256("totalSupply()")[:4]
    }

    # Fetch and decode results
    results = {}
    for func, selector in selectors.items():
        raw_result = fetch_contract_function(contract_address, selector)
        if raw_result:
            if func == "name":
                results[func] = decode_string(raw_result)
            else:
                results[func] = decode_integer(raw_result)

    # Print results
    print("Pool Details:")
    print(f"Name: {results.get('name', 'N/A')}")
    print(f"Actual Supply: {results.get('getActualSupply', 'N/A')}")
    print(f"Total Supply: {results.get('totalSupply', 'N/A')}")


if __name__ == "__main__":
    main()

Pool Details:
Name: 20wstETH-80AAVE
Actual Supply: 751180719628409392048502
Total Supply: 751180719628409392048502


In [1]:
# Checksum

%pip install eth_utils

Note: you may need to restart the kernel to use updated packages.


In [2]:
import re
import eth_utils


def manual_checksum_conversion(address):
    """
    Manual implementation of Ethereum checksum conversion
    """
    # Remove '0x' if present and convert to lowercase
    addr = address.lower().replace("0x", "")

    # Hash the address
    hash_addr = eth_utils.keccak(addr.encode("ascii")).hex()[2:]

    # Convert to checksum
    checksum_addr = "0x"
    for i, char in enumerate(addr):
        # If the corresponding bit in the hash is 1, capitalize the letter
        if int(hash_addr[i], 16) >= 8:
            checksum_addr += char.upper()
        else:
            checksum_addr += char

    return checksum_addr


def library_checksum_conversion(address):
    """
    Convert address to checksum using eth_utils library
    """
    return eth_utils.to_checksum_address(address)


def validate_checksum(address):
    """
    Validate if an address is in correct checksum format
    """
    try:
        return eth_utils.is_checksum_address(address)
    except Exception:
        return False


def main():
    # Test addresses
    addresses = [
        "0xff34b3d4aee8ddcd6f9afffb6fe49bd371b8a357",
        "0x3BfA4F1b5EfECc0B829Eee6C16AaaA4Cb1D34Db4",
    ]

    print("Checksum Conversion Demonstration:")
    print("-" * 50)

    for addr in addresses:
        print(f"\nOriginal Address: {addr}")

        # Manual conversion
        manual_checksum = manual_checksum_conversion(addr)
        print(f"Manual Checksum:  {manual_checksum}")

        # Library conversion
        library_checksum = library_checksum_conversion(addr)
        print(f"Library Checksum: {library_checksum}")

        # Validation
        is_valid_checksum = validate_checksum(library_checksum)
        print(f"Is Valid Checksum: {is_valid_checksum}")

        # Verify both methods produce same result
        print(f"Methods Match: {manual_checksum.lower() == library_checksum.lower()}")


if __name__ == "__main__":
    main()

Checksum Conversion Demonstration:
--------------------------------------------------

Original Address: 0xff34b3d4aee8ddcd6f9afffb6fe49bd371b8a357
Manual Checksum:  0xff34b3D4aee8DdCD6F9AFFfB6fE49BD371b8a357
Library Checksum: 0xFF34B3d4Aee8ddCd6F9AFFFB6Fe49bD371b8a357
Is Valid Checksum: True
Methods Match: True

Original Address: 0x3BfA4F1b5EfECc0B829Eee6C16AaaA4Cb1D34Db4
Manual Checksum:  0x3BFA4F1B5efEcc0b829eEe6c16aAAA4cb1d34DB4
Library Checksum: 0x3BFA4F1B5EfecC0b829eEe6c16aaaA4CB1d34db4
Is Valid Checksum: True
Methods Match: True


In [6]:
import os
from web3 import Web3
from web3.middleware import simple_cache_middleware, retry_middleware


# Uniswap V2 Router address on Sepolia (this is an example, verify the actual router address)
UNISWAP_V2_ROUTER_ADDRESS = "0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D"

# Token addresses
SEPOLIA_ETH_ADDRESS = "0x1caF820bCb6CEFc9deCad36f11fAF4024A9bcF0b"
DAI_ADDRESS = "0xff34b3d4aee8ddcd6f9afffb6fe49bd371b8a357"

# ERC-20 ABI (minimal ABI for token interactions)
ERC20_ABI = [
    {
        "constant": False,
        "inputs": [
            {"name": "_spender", "type": "address"},
            {"name": "_value", "type": "uint256"},
        ],
        "name": "approve",
        "outputs": [{"name": "", "type": "bool"}],
        "type": "function",
    },
    {
        "constant": True,
        "inputs": [{"name": "_owner", "type": "address"}],
        "name": "balanceOf",
        "outputs": [{"name": "balance", "type": "uint256"}],
        "type": "function",
    },
    {
        "constant": True,
        "inputs": [],
        "name": "decimals",
        "outputs": [{"name": "", "type": "uint8"}],
        "type": "function",
    },
]

# Uniswap V2 Router ABI (minimal required functions)
UNISWAP_V2_ROUTER_ABI = [
    {
        "inputs": [
            {"internalType": "uint256", "name": "amountIn", "type": "uint256"},
            {"internalType": "uint256", "name": "amountOutMin", "type": "uint256"},
            {"internalType": "address[]", "name": "path", "type": "address[]"},
            {"internalType": "address", "name": "to", "type": "address"},
            {"internalType": "uint256", "name": "deadline", "type": "uint256"},
        ],
        "name": "swapExactTokensForTokens",
        "outputs": [
            {"internalType": "uint256[]", "name": "amounts", "type": "uint256[]"}
        ],
        "stateMutability": "nonpayable",
        "type": "function",
    }
]


def perform_token_swap(private_key, from_token, to_token, amount_to_swap):
    # Connect to Sepolia testnet (replace with your Infura or Alchemy endpoint)
    w3 = Web3(
        Web3.HTTPProvider(
            "https://sepolia.infura.io/v3/c2277863d49b42909d1a6b452b4d2553"
        )
    )

    # Add PoA middleware for Sepolia
    # Add middleware for Sepolia
    w3.middleware_onion.inject(simple_cache_middleware, layer=0)
    w3.middleware_onion.inject(retry_middleware, layer=1)
    w3.middleware_onion.inject(
        # For Sepolia and other PoA networks
        lambda make_request, w3: {
            "middleware_onion_layer": "inject",
            "make_request": make_request,
        },
        layer=2,
    )

    # Get account from private key
    account = w3.eth.account.privateKeyToAccount(private_key)
    w3.eth.default_account = account.address

    # Create contract instances
    from_token_contract = w3.eth.contract(address=from_token, abi=ERC20_ABI)
    to_token_contract = w3.eth.contract(address=to_token, abi=ERC20_ABI)
    router_contract = w3.eth.contract(
        address=UNISWAP_V2_ROUTER_ADDRESS, abi=UNISWAP_V2_ROUTER_ABI
    )

    # Get token decimals
    from_decimals = from_token_contract.functions.decimals().call()

    # Calculate token amount with decimals
    token_amount = int(amount_to_swap * (10**from_decimals))

    # Check token balance
    balance = from_token_contract.functions.balanceOf(account.address).call()
    if balance < token_amount:
        raise ValueError(
            f"Insufficient balance. Current balance: {balance / (10 ** from_decimals)}"
        )

    # Approve router to spend tokens
    approve_tx = from_token_contract.functions.approve(
        UNISWAP_V2_ROUTER_ADDRESS, token_amount
    ).buildTransaction(
        {
            "from": account.address,
            "nonce": w3.eth.get_transaction_count(account.address),
            "gas": 100000,  # Adjust gas limit as needed
            "gasPrice": w3.eth.gas_price,
        }
    )

    # Sign and send approve transaction
    signed_approve_tx = account.sign_transaction(approve_tx)
    approve_tx_hash = w3.eth.send_raw_transaction(signed_approve_tx.rawTransaction)
    w3.eth.wait_for_transaction_receipt(approve_tx_hash)
    print(f"Tokens approved. Tx hash: {approve_tx_hash.hex()}")

    # Prepare swap transaction
    swap_tx = router_contract.functions.swapExactTokensForTokens(
        token_amount,  # Amount of input tokens to swap
        0,  # Minimum amount of output tokens (set to 0, but in production, calculate this)
        [from_token, to_token],  # Token swap path
        account.address,  # Recipient address
        w3.eth.get_block("latest")["timestamp"] + 300,  # Deadline (5 minutes from now)
    ).buildTransaction(
        {
            "from": account.address,
            "nonce": w3.eth.get_transaction_count(account.address),
            "gas": 200000,  # Adjust gas limit as needed
            "gasPrice": w3.eth.gas_price,
        }
    )

    # Sign and send swap transaction
    signed_swap_tx = account.sign_transaction(swap_tx)
    swap_tx_hash = w3.eth.send_raw_transaction(signed_swap_tx.rawTransaction)

    # Wait for transaction receipt
    tx_receipt = w3.eth.wait_for_transaction_receipt(swap_tx_hash)
    print(f"Swap successful! Transaction hash: {swap_tx_hash.hex()}")

    return tx_receipt


def main():
    # IMPORTANT: NEVER hardcode your private key in production!
    # Use environment variables or secure key management
    # private_key = os.getenv("SEPOLIA_PRIVATE_KEY")
    private_key = "4c77e4ca383bad8216dea0d671909446169de7aa3529a22596ecfcb9e845214f"

    if not private_key:
        raise ValueError("Please set SEPOLIA_PRIVATE_KEY environment variable")

    try:
        # Example: Swap 1 sepoliaETH for DAI
        receipt = perform_token_swap(
            private_key, SEPOLIA_ETH_ADDRESS, DAI_ADDRESS, 1  # Amount to swap
        )
        print("Swap details:", receipt)

    except Exception as e:
        print(f"An error occurred: {e}")


if __name__ == "__main__":
    main()

# IMPORTANT NOTES:
# 1. Replace 'YOUR-PROJECT-ID' with your actual Infura or Alchemy Sepolia endpoint
# 2. Set your private key as an environment variable
# 3. Ensure you have sufficient testnet ETH for gas fees
# 4. Verify token addresses and router address for the Sepolia network
# 5. This script is for educational purposes. Always test thoroughly on testnets

ImportError: cannot import name 'simple_cache_middleware' from 'web3.middleware' (c:\Users\msadi\anaconda3\envs\newllama\Lib\site-packages\web3\middleware\__init__.py)

In [15]:
from web3 import Web3

# Configuration
sepolia_rpc_url = "https://sepolia.infura.io/v3/c2277863d49b42909d1a6b452b4d2553"  # Replace with your Sepolia RPC URL
web3 = Web3(Web3.HTTPProvider(sepolia_rpc_url))
if not web3.is_connected():
    raise Exception("Failed to connect to Sepolia network")

# Wallet setup
private_key = "4c77e4ca383bad8216dea0d671909446169de7aa3529a22596ecfcb9e845214f"  # Replace with your private key
wallet_address = (
    "0x1caF820bCb6CEFc9deCad36f11fAF4024A9bcF0b"  # Replace with your wallet address
)

# Contract addresses
uniswap_router_address = web3.to_checksum_address(
    "0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D"
)  # Replace with Uniswap Router address on Sepolia
eth_token_address = web3.to_checksum_address(
    "0x1caF820bCb6CEFc9deCad36f11fAF4024A9bcF0b"  # ETH (WETH equivalent)
)
dai_token_address = web3.to_checksum_address(
    "0xff34b3d4aee8ddcd6f9afffb6fe49bd371b8a357"
)  # DAI

# Swap parameters
eth_amount_in = web3.to_wei(0.1, "ether")  # Amount of ETH to swap
min_dai_out = web3.to_wei(1, "ether")  # Minimum amount of DAI to receive
deadline = int(web3.eth.get_block("latest")["timestamp"]) + 300  # 5-minute deadline

# Uniswap Router ABI (partial, replace with full ABI if necessary)
router_abi = [
    {
        "inputs": [
            {"internalType": "uint256", "name": "amountOutMin", "type": "uint256"},
            {"internalType": "address[]", "name": "path", "type": "address[]"},
            {"internalType": "address", "name": "to", "type": "address"},
            {"internalType": "uint256", "name": "deadline", "type": "uint256"},
        ],
        "name": "swapExactETHForTokens",
        "outputs": [
            {"internalType": "uint256[]", "name": "amounts", "type": "uint256[]"}
        ],
        "stateMutability": "payable",
        "type": "function",
    }
]

# Setup Uniswap Router contract
uniswap_router = web3.eth.contract(address=uniswap_router_address, abi=router_abi)


# Build the transaction
def build_eth_to_dai_transaction():
    current_gas_price = web3.eth.gas_price
    gas_price = current_gas_price + web3.to_wei("2", "gwei")
    nonce = web3.eth.get_transaction_count(wallet_address)
    tx = uniswap_router.functions.swapExactETHForTokens(
        min_dai_out, [eth_token_address, dai_token_address], wallet_address, deadline
    ).build_transaction(
        {
            "from": wallet_address,
            "value": eth_amount_in,  # Amount of ETH sent
            "gas": 200000,  # Adjust based on estimation
            "gasPrice": gas_price,  # Adjust gas price
            "nonce": nonce,
        }
    )
    return tx


# Sign and send the transaction
def send_transaction(tx):
    signed_tx = web3.eth.account.sign_transaction(tx, private_key=private_key)
    tx_hash = web3.eth.send_raw_transaction(signed_tx.raw_transaction)
    print(f"Transaction sent! Hash: {web3.to_hex(tx_hash)}")
    return tx_hash


# Main script
if __name__ == "__main__":
    try:
        print("Building transaction...")
        transaction = build_eth_to_dai_transaction()
        print("Sending transaction...")
        tx_hash = send_transaction(transaction)
        print("Waiting for receipt...")
        receipt = web3.eth.wait_for_transaction_receipt(tx_hash)
        print(f"Transaction successful! Receipt: {receipt}")
    except Exception as e:
        print(f"An error occurred: {e}")

Building transaction...
Sending transaction...
Transaction sent! Hash: 0x51b93bf69494dde812fe9a400f3d38878b803579a26907458e523de6ea82d04f
Waiting for receipt...
Transaction successful! Receipt: AttributeDict({'blockHash': HexBytes('0x861c86b2f8ae141ec1389f9c6032c884c8f1e8ed05c79eb4e61272f205d6906b'), 'blockNumber': 7176918, 'contractAddress': None, 'cumulativeGasUsed': 186708, 'effectiveGasPrice': 6392661607, 'from': '0x1caF820bCb6CEFc9deCad36f11fAF4024A9bcF0b', 'gasUsed': 22824, 'logs': [], 'logsBloom': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

from web3 import Web3

# Configuration

sepolia_rpc_url = "https://sepolia.infura.io/v3/YOUR_INFURA_PROJECT_ID" # Replace with your Sepolia RPC URL
web3 = Web3(Web3.HTTPProvider(sepolia_rpc_url))
if not web3.isConnected():
raise Exception("Failed to connect to Sepolia network")

# Wallet setup

private_key = "0xYourPrivateKey" # Replace with your private key
wallet_address = "0xYourWalletAddress" # Replace with your wallet address

# Contract addresses

uniswap_router_address = "0xYourUniswapRouterAddress" # Replace with Uniswap Router address on Sepolia
eth_token_address = "0x1caF820bCb6CEFc9deCad36f11fAF4024A9bcF0b" # ETH (WETH equivalent)
dai_token_address = "0xff34b3d4aee8ddcd6f9afffb6fe49bd371b8a357" # DAI

# Swap parameters

eth_amount_in = web3.toWei(0.1, "ether") # Amount of ETH to swap
min_dai_out = web3.toWei(1, "ether") # Minimum amount of DAI to receive
deadline = int(web3.eth.get_block("latest")["timestamp"]) + 300 # 5-minute deadline

# Uniswap Router ABI (partial, replace with full ABI if necessary)

router_abi = [
{
"inputs": [
{"internalType": "uint256", "name": "amountOutMin", "type": "uint256"},
{"internalType": "address[]", "name": "path", "type": "address[]"},
{"internalType": "address", "name": "to", "type": "address"},
{"internalType": "uint256", "name": "deadline", "type": "uint256"},
],
"name": "swapExactETHForTokens",
"outputs": [{"internalType": "uint256[]", "name": "amounts", "type": "uint256[]"}],
"stateMutability": "payable",
"type": "function",
}
]

# Setup Uniswap Router contract

uniswap_router = web3.eth.contract(address=uniswap_router_address, abi=router_abi)

# Build the transaction

def build_eth_to_dai_transaction():
nonce = web3.eth.get_transaction_count(wallet_address)
tx = uniswap_router.functions.swapExactETHForTokens(
min_dai_out,
[eth_token_address, dai_token_address],
wallet_address,
deadline
).buildTransaction({
"from": wallet_address,
"value": eth_amount_in, # Amount of ETH sent
"gas": 200000, # Adjust based on estimation
"gasPrice": web3.toWei("5", "gwei"), # Adjust gas price
"nonce": nonce,
})
return tx

# Sign and send the transaction

def send_transaction(tx):
signed_tx = web3.eth.account.sign_transaction(tx, private_key=private_key)
tx_hash = web3.eth.send_raw_transaction(signed_tx.rawTransaction)
print(f"Transaction sent! Hash: {web3.toHex(tx_hash)}")
return tx_hash

# Main script

if **name** == "**main**":
try:
print("Building transaction...")
transaction = build_eth_to_dai_transaction()
print("Sending transaction...")
tx_hash = send_transaction(transaction)
print("Waiting for receipt...")
receipt = web3.eth.wait_for_transaction_receipt(tx_hash)
print(f"Transaction successful! Receipt: {receipt}")
except Exception as e:
print(f"An error occurred: {e}")
